In [1]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
print(f'Current working directory is {os.getcwd()}')

Current working directory is /Users/drewrichard/Documents/1 projects/nss/nss_projects/accre-carbonara/notebooks


In [5]:
# Import data
jobs = pd.read_csv("../data/fullsample.csv",
                    nrows = 1000
                    )
ce5 = pd.read_csv('../data/ce5_unresponsive.csv')
ce6 = pd.read_csv('../data/ce6_unresponsive.csv')
jobs = jobs[jobs['END'] != 'Unknown']

# Convert dates to datetime objects
jobs['END'] = pd.to_datetime(jobs['END'])
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'])

# String manipulation of memory columns
jobs['USEDMEM'] = jobs['USEDMEM'].str[:-1]
jobs.insert(5, 'Mc_Mn', jobs['REQMEM'].str[-2:])
jobs['REQMEM']  = jobs['REQMEM'].str[:-2]

# Convert to numeric values for easier manipulation, and create difference col
jobs['USEDMEM'] = pd.to_numeric(jobs['USEDMEM'])
jobs['REQMEM'] = pd.to_numeric(jobs['REQMEM'])
jobs['DIFFMEM'] = jobs['REQMEM'] - jobs['USEDMEM']

# Create boolean column for status = COMPLETE
jobs.insert(2, 'COMPLETE', jobs['STATE'] == 'COMPLETED')
jobs['COMPLETE'] = jobs['COMPLETE'].astype(int)

# Create columns to assign failure and concat into one df
ce56 = pd.concat([ce5, ce6])
jobs['FAILED'] = 0
ce56['FAILED'] = 1
jobs_ce56 = pd.concat([jobs, ce56])

# Eliminate milliseconds in desired datetime column 'END' 
jobs_ce56['END'] = pd.to_datetime(jobs_ce56['END']).dt.floor('s')
jobs_ce56.head(5)

,JOBID,STATE,COMPLETE,BEGIN,END,REQMEM,Mc_Mn,USEDMEM,REQTIME,USEDTIME,...,CPUS,PARTITION,EXITCODE,DIFFMEM,FAILED,USER,RETRY,TIME,RETURNCODE,COMMAND
1,30853133,COMPLETED,1.0,2021-08-06 11:36:09,2021-09-05 11:36:32,262144.0,Mn,20604.62,30-00:00:00,30-00:00:23,...,1.0,cgw-platypus,0:0,241539.38,0,NaN,NaN,NaN,NaN,NaN
2,30858137,COMPLETED,1.0,2021-08-06 19:04:39,2021-09-05 19:04:53,204800.0,Mn,57553.77,30-00:00:00,30-00:00:14,...,32.0,cgw-tbi01,0:0,147246.23,0,NaN,NaN,NaN,NaN,NaN
3,30935078,COMPLETED,1.0,2021-08-09 16:52:51,2021-09-07 20:52:55,65536.0,Mn,20577.96,29-04:00:00,29-04:00:04,...,8.0,cgw-platypus,0:0,44958.04,0,NaN,NaN,NaN,NaN,NaN
4,31364111_2,COMPLETED,1.0,2021-08-17 07:45:07,2021-09-10 16:45:24,16384.0,Mn,9733.43,24-09:00:00,24-09:00:17,...,1.0,production,0:0,6650.57,0,NaN,NaN,NaN,NaN,NaN
5,31364111_3,COMPLETED,1.0,2021-08-17 07:45:07,2021-09-06 16:17:34,16384.0,Mn,9708.04,24-09:00:00,20-08:32:27,...,1.0,production,0:0,6675.96,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Parameters
x_mins = 10

# Identify indicator rows
indicator_rows = jobs_ce56[jobs_ce56['FAILED'] == 1]

# Initialize an empty DataFrame to store results
result_df = pd.DataFrame()

# Iterate through each indicator row
for _, row in indicator_rows.iterrows():
    time_threshold = row['END'] - timedelta(minutes=x_mins)
    filtered_rows = jobs_ce56[(jobs_ce56['END'] >= time_threshold) & (jobs_ce56['END'] < row['END'])]
    result_df = pd.concat([result_df, filtered_rows])

display(result_df)

In [ ]:
# # Parameters
# x_mins = 10

# # Find all rows with the indicator set to 1
# indicator_times = jobs_ce56.loc[jobs_ce56['FAILED'] == 1, 'END']

# # Create a mask for rows within x minutes before any indicator time
# time_windows = (jobs_ce56['END'].values[:, None] >= (indicator_times - pd.Timedelta(minutes=x_mins)).values) & \
#                (jobs_ce56['END'].values[:, None] < indicator_times.values)

# # Combine the mask for all indicator times
# mask = time_windows.any(axis=1)

# # Filter rows
# result_df = jobs_ce56[mask]

# display(result_df)

In [ ]:
result_df.sort_values('END')

In [ ]:
# row_counts = time_windows.sum(axis=0)

# # Combine results into a DataFrame for clarity
# rowcounts_df = pd.DataFrame({
#     'indicator_time': indicator_times.values,
#     'row_count': row_counts
# })
# display(rowcounts_df.sort_values('row_count'))